In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import npts

In [ ]:
data = pd.read_csv('http://stanford.edu/class/ee103/data/returns.txt', index_col=0, parse_dates=[0])
data_used = data.iloc[:, :10]

In [ ]:
baseline = npts.Baseline(npts.QuarterOfYear(lambdas=np.logspace(-4,4,10)),
                               npts.BDayOfQuarter(lambdas=np.logspace(-4,4,10)), verbose=True)
baseline.fit(data_used)

In [ ]:
def plot_RMSE(baseline, cost_dict, title):
    fig = plt.figure(figsize=(8,4))
    cax = plt.hexbin(*np.array([[*np.log10(k), np.sqrt(v)] for k, v 
                          in cost_dict.items()]).T,
              gridsize=20)
    axc = fig.colorbar(cax, ax=fig.gca(), shrink=.7)
    plt.plot(*np.log10(baseline.best_lambda), 'ro', markersize=10)
    plt.xlabel('log10(λ_quarter)')
    plt.ylabel('log10(λ_day)')
    plt.title(title)
    
plot_RMSE(weekly_seasonal, weekly_seasonal.val_costs, 'test')


In [ ]:
def plot_2d_model(theta):
    
    plt.figure()
    plt.plot(theta)
    plt.xlabel('hours since start of year')

    fig = plt.figure()
    cax = plt.imshow(theta.reshape((66,4)).T, 
                     aspect='auto',origin='lower',interpolation='gaussian')
    axc = fig.colorbar(cax, ax=fig.gca(), shrink=.7)
    plt.xlabel('dayofquarter')
    plt.ylabel('quarter')

plot_2d_model(weekly_seasonal.theta[:,0])


In [ ]:

plt.plot(weekly_seasonal.theta)

In [ ]:
# ### Error baseline 

# In[71]:

residuals = data_used - baseline.predict(data_used.index).values


# In[82]:

import itertools
outer_product_col = list(itertools.combinations_with_replacement(residuals.columns, 2))
residual_squares = pd.DataFrame(columns=outer_product_col)

for col in residual_squares.columns:
    residual_squares[col] = residuals[col[0]] * residuals[col[1]]


# In[83]:

residual_squares


# In[89]:

residual_squares.shape


# In[94]:

cov_baseline = npts.Baseline(npts.QuarterOfYear(lambdas=np.logspace(-4,4,10)),
                               npts.BDayOfQuarter(lambdas=np.logspace(-4,4,10)), verbose=True)
cov_baseline.fit(residual_squares)


# In[95]:

def plot_RMSE(baseline, cost_dict, title):
    fig = plt.figure(figsize=(8,4))
    cax = plt.hexbin(*np.array([[*np.log10(k), np.sqrt(v)] for k, v 
                          in cost_dict.items()]).T,
              gridsize=20)
    axc = fig.colorbar(cax, ax=fig.gca(), shrink=.7)
    plt.plot(*np.log10(baseline.best_lambda), 'ro', markersize=10)
    plt.xlabel('log10(λ_quarter)')
    plt.ylabel('log10(λ_day)')
    plt.title(title)
    
plot_RMSE(cov_baseline, cov_baseline.val_costs, 'test')


# In[96]:

plt.plot(cov_baseline.theta)


# In[122]:

t = 5

cov = pd.DataFrame(index=data_used.columns,
             columns=data_used.columns,dtype=float)

for i,pair in enumerate(residual_squares.columns):
    cov.loc[pair[0], pair[1]] = cov_baseline.theta[t,i]
    cov.loc[pair[1], pair[0]] = cov_baseline.theta[t,i]
    
plt.imshow(cov.values)


# In[ ]:
